In [1]:
url='http://www.bsp.gov.ph/statistics/spei_new/tab34_inf.htm'

### For next time you use this:

Since programmer who updates PH inflation data appears to change the td class associated with the data points regularly, easier to manually get latest data point than to extract the table all over again with new patterns

Check below for code labeled ***UPDATED CODE***

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [10]:
response=requests.get(url)
soup=BeautifulSoup(response.text,'lxml')
inf_table=soup.find("table")
inf_table=inf_table.find_all('td')
pattern=re.compile("xl7228912")
inflation=[]
for item in soup.find_all("td",pattern):
    inflation.append(item)

In [12]:
inflation_data=[]
pattern_infl=re.compile(">(\d\D\d)")
for item in inflation:
    try:
        infl=float(pattern_infl.search(str(item)).group(1))
    except:
        infl=None
    inflation_data.append(infl)

In [16]:
#extract years as columns
years=[]
#old code#pattern=re.compile("xl749132")
pattern=re.compile("xl6928912")
for item in soup.find_all("td",pattern):
    years.append(item)
pattern_years=re.compile("</span>(\d+)")
years_columns=[pattern_years.search(str(x)).group(1) for x in years]

In [25]:
#reshape using numpy
inflation_data=np.array(inflation_data).reshape(12,len(years))

#change to pandas dataframe
inflation_data=pd.DataFrame(inflation_data,columns=years_columns)

#add month column
month=pd.DataFrame(np.arange(1,13),columns=['Month'])
inflation_data=pd.concat([month,inflation_data],axis=1)
inflation_data.set_index('Month',inplace=True)
inflation_data

,2016,2017,2018,2019,2020
Month,,,,,
1,0.7,2.5,3.4,4.4,2.9
2,0.5,3.1,3.8,3.8,2.6
3,0.6,3.1,4.3,3.3,2.5
4,0.7,3.2,4.5,3,2.2
5,0.9,2.9,4.6,3.2,2.1
6,1.3,2.5,5.2,2.7,2.5
7,1.3,2.4,5.7,2.4,2.7
8,1.3,2.6,6.4,1.7,None
9,1.7,3,6.7,0.9,None


In [135]:
# Stack to make it easier to append to historical dataframe
infl=pd.DataFrame(inflation_data.transpose().stack())
infl.reset_index(inplace=True)
infl.columns=['Year','Month','Inflation']
infl

,Year,Month,Inflation
0,2016,1,0.7
1,2016,2,0.5
2,2016,3,0.6
3,2016,4,0.7
4,2016,5,0.9
5,2016,6,1.3
6,2016,7,1.3
7,2016,8,1.3
8,2016,9,1.7
9,2016,10,1.8


In [136]:
infl.to_csv("PH_inflation.csv")

## Find last available data point

In [139]:
# Find last available data point
infl=pd.read_csv("PH_inflation.csv",index_col=0)
infl.tail()

,Year,Month,Inflation
49,2020,2,2.6
50,2020,3,2.5
51,2020,4,2.2
52,2020,5,2.1
53,2020,6,2.5


In [143]:
last_year=infl.tail(1)['Year'].iloc[0]
last_month=infl.tail(1)['Month'].iloc[0]

## Update if data point is not yet in historical

In [157]:
# Load historical
hist_infl=pd.read_csv("PH_inflation_historical.csv",index_col=0)
last_histdata_point=hist_infl.tail(1)
last_histdata_point

,Year,Month,Inflation
53,2020,6,2.5


In [158]:
#number of data points to be updated
datapoints=12*(int(last_year)-last_histdata_point['Year'].iloc[0])-last_histdata_point['Month'].iloc[0]+last_month

In [159]:
#append to historical record and output this updated inflation table to csv
updated_infl=hist_infl.append(infl.tail(datapoints))
updated_infl.tail()

,Year,Month,Inflation
85,2020,2,2.6
86,2020,3,2.5
87,2020,4,2.2
52,2020,5,2.1
53,2020,6,2.5


In [160]:
updated_infl.to_csv("PH_inflation_historical.csv")

*** UPDATED CODE ***

Since programmer who updates PH inflation data appears to change the td class associated with the data points regularly, easier to manually get latest data point than to extract the table all over again with new patterns

In [11]:
import pandas as pd
infl=pd.read_csv("PH_inflation_historical.csv")
infl.tail()

,Year,Month,Inflation
86,2020,3,2.5
87,2020,4,2.2
88,2020,5,2.1
89,2020,6,2.5
90,2020,7,2.7


In [5]:
# Inputs
year_input=2020
month_input=7
inflation_input=2.7

In [6]:
new_infl=pd.DataFrame([int(year_input),int(month_input),inflation_input]).transpose()
new_infl.columns=['Year','Month','Inflation']
new_infl

,Year,Month,Inflation
0,2020.0,7.0,2.7


In [12]:
import numpy as np
updated_infl=infl.append(new_infl)
cols=['Year','Month']
updated_infl[cols]=updated_infl[cols].applymap(np.int64)
updated_infl.tail()

,Year,Month,Inflation
87,2020,4,2.2
88,2020,5,2.1
89,2020,6,2.5
90,2020,7,2.7
0,2020,7,2.7


In [16]:
updated_infl.to_csv("PH_inflation_historical.csv",index=False)

## Extract PH GDP

In [58]:
import pandas as pd
import numpy as np
import re

In [8]:
# wont use this for now. cant find way to show last Q1 column (even with pd.read_excel() and requests)
url="https://psa.gov.ph/sites/default/files/01Summary_2018PSNA_Qrt_4.xlsx"

In [17]:
# temporary measure: manually download excel then save as csv. for some reason, last column shows up this way
df=pd.read_csv("01Summary_2018PSNA_Qrt_4.csv")
df.iloc[[142],]

,National Accounts of the Philippines,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81
142,Gross Domestic Product,2.4,3.3,3.1,3.3,3.9,3.7,2.7,4.6,4.6,...,6.4,5.7,5.4,6.3,6.7,-0.2,NaN,NaN,NaN,NaN


In [59]:
quarters=df.iloc[[122],].transpose()[1:].reset_index()
years=df.iloc[[121]].transpose().dropna().reset_index()

In [60]:
#replace "Unnamed" to blank
quarters['index']=quarters['index'].str.replace(r'Unnamed: ','').astype(int)
years['index']=years['index'].str.replace(r'Unnamed: ','').astype(int)

In [61]:
quarters.columns=['label','quarter']
years.columns=['label','year']

In [62]:
quarters.head()

,label,quarter
0,1,Q1
1,2,Q2
2,3,Q3
3,4,Q4
4,5,Q1


In [63]:
years.head()

,label,year
0,1,2000 - 2001
1,5,2001 - 2002
2,9,2002 - 2003
3,13,2003 - 2004
4,17,2004 - 2005


In [83]:
label=pd.merge_asof(quarters,years,on='label')
label=label[['year','quarter']]
label.head()

,quarter,year
0,Q1,2000 - 2001
1,Q2,2000 - 2001
2,Q3,2000 - 2001
3,Q4,2000 - 2001
4,Q1,2001 - 2002


In [97]:
gdpgrowth=df.iloc[[142],].transpose()[1:].reset_index()[[142]]
gdpgrowth.columns=['GDP']
gdpgrowth.head()

,GDP
0,2.4
1,3.3
2,3.1
3,3.3
4,3.9


In [102]:
GDP=pd.concat([label,gdpgrowth],axis=1)
GDP.dropna(inplace=True)

In [103]:
GDP

,quarter,year,GDP
0,Q1,2000 - 2001,2.4
1,Q2,2000 - 2001,3.3
2,Q3,2000 - 2001,3.1
3,Q4,2000 - 2001,3.3
4,Q1,2001 - 2002,3.9
...,...,...,...
72,Q1,2018 - 2019,5.7
73,Q2,2018 - 2019,5.4
74,Q3,2018 - 2019,6.3
75,Q4,2018 - 2019,6.7


In [104]:
GDP.to_csv("PH_GDP.csv")

## Update the latest GDP growth then append

In [61]:
ph_gdp=pd.read_csv("PH_GDP.csv",index_col=0)

In [63]:
ph_gdp.tail()

,quarter,year,GDP
72,Q1,2018 - 2019,5.7
73,Q2,2018 - 2019,5.4
74,Q3,2018 - 2019,6.3
75,Q4,2018 - 2019,6.7
76,Q1,2019 - 2020,-0.2


In [ ]:
# Inputs
quarter_input='Q2'
year_input='2019-2020'
growth_input=